In [2]:
import peewee as pw
import pandas as pd
import numpy as np
from structure import Consts, Functions, Solves, database_connection
from solve import default_values, const_keys, path

In [3]:
def df_from_comsol(name):
    with open(path + r'\Batch\io\Data.csv', 'r') as file:
        lines = file.readlines()
    lines = lines[4:]

    lines[:] = (line.replace('\n', '') for line in lines[:])
    lines[0] = lines[0].replace('% ', '')

    data = [line.split(',') for line in lines]
    return pd.DataFrame(data=data[1:], columns=data[0]).astype(float)


In [4]:
df = df_from_comsol('test')
df.head()

,Ke,KH,Kr,Kdisp,KqH,Ks,Kd,Kc,Kp,KrD,...,Q,DH,QHD,QHH,prod,Qm,DHp,QH,D,QD
0,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09,2000.0,3.0,0.05,1.0,0.001,1.000000e+09,...,1.0,1.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0
1,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09,2000.0,3.0,0.05,1.0,0.001,1.000000e+09,...,1.0,1.0,0.0,0.0,2.000000e-12,0.0,0.0,0.0,0.0,0.0
2,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09,2000.0,3.0,0.05,1.0,0.001,1.000000e+09,...,1.0,1.0,0.0,0.0,4.000000e-12,0.0,0.0,0.0,0.0,0.0
3,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09,2000.0,3.0,0.05,1.0,0.001,1.000000e+09,...,1.0,1.0,0.0,0.0,8.000000e-12,0.0,0.0,0.0,0.0,0.0
4,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09,2000.0,3.0,0.05,1.0,0.001,1.000000e+09,...,1.0,1.0,0.0,0.0,1.600000e-11,0.0,0.0,0.0,0.0,0.0


In [44]:
nonconst_keys = [i for i in df.columns if i not in const_keys]
a = []
for const_values, frame in df.groupby(by=const_keys):
    # Solve
    solve_q = 'solve1'

    print(solve_q)

    # Consts
    zipped = zip(const_keys, const_values)
    consts_q = [{
        'solve': solve_q,
        'key': key,
        'value': value,
    } for key,
                value in zipped]

    print(consts_q)

    # print(nonconst_keys)

    # print(name)
    func_frame = frame[nonconst_keys]
    # func_frame = func_frame.set_index('Time')
    # func_frame = func_frame.unstack().reset_index()
    # func_frame = func_frame.rename(columns={
    #     'Time': 'time',
    #     'level_0': 'function',
    #     0: 'value',
    # })

    # for num,i in func_frame.iterrows():
    #     func_data = dict(i)
    #     func_data.update({'solve':solve_q})
    #     # data_q.append(func_data)
    a.append(func_frame)

solve1
[{'solve': 'solve1', 'key': 'Ke', 'value': 1000000000.0}, {'solve': 'solve1', 'key': 'KH', 'value': 1000000000.0}, {'solve': 'solve1', 'key': 'Kr', 'value': 1000000000.0}, {'solve': 'solve1', 'key': 'Kdisp', 'value': 1000000000.0}, {'solve': 'solve1', 'key': 'KqH', 'value': 2000.0}, {'solve': 'solve1', 'key': 'Ks', 'value': 3.0}, {'solve': 'solve1', 'key': 'Kd', 'value': 0.05}, {'solve': 'solve1', 'key': 'Kc', 'value': 1.0}, {'solve': 'solve1', 'key': 'Kp', 'value': 0.001}, {'solve': 'solve1', 'key': 'KrD', 'value': 1000000000.0}, {'solve': 'solve1', 'key': 'Kph', 'value': 1e-05}, {'solve': 'solve1', 'key': 'light', 'value': 0.0}]
solve1
[{'solve': 'solve1', 'key': 'Ke', 'value': 1000000000.0}, {'solve': 'solve1', 'key': 'KH', 'value': 1000000000.0}, {'solve': 'solve1', 'key': 'Kr', 'value': 1000000000.0}, {'solve': 'solve1', 'key': 'Kdisp', 'value': 1000000000.0}, {'solve': 'solve1', 'key': 'KqH', 'value': 2000.0}, {'solve': 'solve1', 'key': 'Ks', 'value': 3.0}, {'solve': 'solv

In [45]:
b=a[2]
b

,Time,Q,DH,QHD,QHH,prod,Qm,DHp,QH,D,QD
311,0.000000e+00,1.000000,1.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
312,7.812500e-10,0.999984,0.999984,0.000000e+00,0.000000e+00,7.812378e-15,0.000016,0.000016,1.907229e-10,1.907229e-10,1.490023e-19
313,1.366070e-09,0.999973,0.999973,2.300638e-19,2.300635e-19,1.366064e-14,0.000027,0.000027,6.272103e-10,6.272103e-10,5.157971e-19
314,2.535709e-09,0.999949,0.999949,1.568128e-17,1.568125e-17,2.537189e-14,0.000051,0.000051,3.634590e-09,3.634590e-09,4.766740e-18
315,4.874987e-09,0.999903,0.999903,1.579405e-15,1.579398e-15,5.032612e-14,0.000097,0.000097,2.585468e-08,2.585468e-08,6.524214e-17
...,...,...,...,...,...,...,...,...,...,...,...
670,1.916527e-04,0.403306,0.115557,3.071314e-01,2.878140e-01,2.878747e-01,0.000968,0.000968,7.157758e-04,5.294404e-04,6.506032e-05
671,1.938447e-04,0.401890,0.113535,3.079539e-01,2.884213e-01,2.884822e-01,0.000958,0.000958,7.110558e-04,5.233532e-04,6.587498e-05
672,1.960367e-04,0.400503,0.111555,3.087616e-01,2.890146e-01,2.890758e-01,0.000948,0.000948,7.064280e-04,5.173618e-04,6.668895e-05
673,1.982287e-04,0.399143,0.109616,3.095549e-01,2.895945e-01,2.896560e-01,0.000938,0.000938,7.018904e-04,5.114637e-04,6.750224e-05


In [46]:

with database_connection:
    solve = Solves.create(name='test', description=None)
    const = Consts.create(solve=solve, key='T', value=1)
    data = Functions.create(solve = solve,data = b.to_json(index=True))
    

In [47]:
import json
pd.read_json
with database_connection:
    c= Functions.select()

    for i in c:
        d=i.data
        


In [49]:
e = pd.read_json(d)
e

,Time,Q,DH,QHD,QHH,prod,Qm,DHp,QH,D,QD
311,0.000000e+00,1.000000,1.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
312,8.000000e-10,0.999984,0.999984,0.000000e+00,0.000000e+00,0.000000,0.000016,0.000016,2.000000e-10,2.000000e-10,1.490023e-19
313,1.400000e-09,0.999973,0.999973,2.300638e-19,2.300635e-19,0.000000,0.000027,0.000027,6.000000e-10,6.000000e-10,5.157971e-19
314,2.500000e-09,0.999949,0.999949,1.568128e-17,1.568125e-17,0.000000,0.000051,0.000051,3.600000e-09,3.600000e-09,4.766740e-18
315,4.900000e-09,0.999903,0.999903,0.000000e+00,0.000000e+00,0.000000,0.000097,0.000097,2.590000e-08,2.590000e-08,6.524214e-17
...,...,...,...,...,...,...,...,...,...,...,...
670,1.916527e-04,0.403306,0.115557,3.071314e-01,2.878140e-01,0.287875,0.000968,0.000968,7.157758e-04,5.294404e-04,6.506030e-05
671,1.938447e-04,0.401890,0.113535,3.079539e-01,2.884213e-01,0.288482,0.000958,0.000958,7.110558e-04,5.233532e-04,6.587500e-05
672,1.960367e-04,0.400503,0.111555,3.087616e-01,2.890146e-01,0.289076,0.000948,0.000948,7.064280e-04,5.173618e-04,6.668900e-05
673,1.982287e-04,0.399143,0.109616,3.095549e-01,2.895945e-01,0.289656,0.000938,0.000938,7.018904e-04,5.114637e-04,6.750220e-05


In [8]:
def distribute(df: pd.DataFrame):
    condition = lambda i: ('K' in i) or ('light' in i)
    consts = [i for i in df.columns if condition(i)]
    nonconsts = [i for i in df.columns if not condition(i)]
    combs = df.drop_duplicates(subset=consts)[consts]
    combs = combs.reset_index(drop=True)
    notes = []
    for num, comb in combs.iterrows():

        df_filtered = df
        for key, value in comb.items():
            df_filtered = df_filtered[df_filtered[key] == value]

        df_filtered = df_filtered.reset_index(drop=True)
        const_vals = dict(zip(comb.keys(), comb.values))
        notes.append((const_vals, df_filtered[nonconsts]))
    return notes

In [6]:
# def save(name=None, desc=None):
#     if name is None:
#         while name != 'q':
#             name = input(f'Set {name=}, to quit - q:')

#     if desc is None:
#         while desc != 'q':
#             desc = input(f'Set {desc=}, to quit - q:')

#     with database_connection:
#         q = Solves(name = name,description=desc)
#         q.save()


In [ ]:
with database_connection:
    solve = Solves.create(name='test', description=None)

    for key, value in a[1][0].items():
        coef = Coefs.create(solve=solve, key=key, value=value)


In [24]:
# Series
[{'solve': '1', 'key': key, 'value': value} for key, value in b.items()]


[{'solve': '1', 'key': 'Ke', 'value': 1000000000.0},
 {'solve': '1', 'key': 'KH', 'value': 1000000000.0},
 {'solve': '1', 'key': 'Kr', 'value': 1000000000.0},
 {'solve': '1', 'key': 'Kdisp', 'value': 1000000000.0},
 {'solve': '1', 'key': 'KqH', 'value': 2000.0},
 {'solve': '1', 'key': 'Ks', 'value': 3.0},
 {'solve': '1', 'key': 'Kd', 'value': 0.05},
 {'solve': '1', 'key': 'Kc', 'value': 1.0},
 {'solve': '1', 'key': 'Kp', 'value': 0.001},
 {'solve': '1', 'key': 'KrD', 'value': 1000000000.0},
 {'solve': '1', 'key': 'Kph', 'value': 1e-05},
 {'solve': '1', 'key': 'light', 'value': 0.0}]